In [26]:
%matplotlib inline

import pandas as pd
import numpy as np

from ipywidgets import interact
from bokeh.io import push_notebook  ,output_notebook , show
from bokeh.plotting import figure
from bokeh.models.widgets import Select 
from bokeh.models import HoverTool, ColumnDataSource , CheckboxGroup , CustomJS
from bokeh.layouts import widgetbox , row , layout


store = pd.HDFStore('../../data/processed/orbf_benin.h5')
data = store['data']
store.close()

Full downloaded data has 564549 indicator/facility/months data points. We limit this data set to only data points with a validation, which reduces the data set to 434292 indicator/facility/months, representing 77% of the complete data.

## High Level Data description

In [14]:
def make_timeline_data(data) :
    out = len(data)
    data['correction'] = (data.indicator_claimed_value == data.indicator_verified_value)
    
    perc_right = sum(data['correction']) / len(data)
    
    return pd.DataFrame([[out, perc_right]])

timeline_data = data.groupby(['entity_name' , 'entity_type_id', 'entity_type' ,'period']).apply(make_timeline_data).reset_index()
timeline_data.columns = ['entity_name', 'entity_type_id' , 'entity_type' ,'date' , 'level_1' , 'value' , 'percent_right']

In [15]:
def make_fac_desc(data) :
    start_reporting = min(data.date)
    n_values = len(data)
    entity_type = data.entity_type_id.unique()
    data['correction'] = (data.indicator_claimed_value == data.indicator_verified_value)
    
    perc_right = sum(data['correction']) / len(data)
    
    return pd.DataFrame([[start_reporting ,n_values , entity_type[0] , perc_right]])

fac_desc = data.groupby('entity_name').apply(make_fac_desc).reset_index()
fac_desc.columns = ['entity_name' , 'level_1' ,'date_start' , 'n_values' , 'entity_type' , 'percent_right']
result = fac_desc.sort_values(['entity_type' ,'date_start', 'n_values' , 'entity_name'], ascending=[ 0 , 0 , 1 , 0])

In [16]:
timeline_data.entity_name_cat = data.entity_name.astype('category').cat.set_categories(result['entity_name'], ordered=True)

In [17]:
output_notebook()
colormap = ["#444444", "#a6cee3", "#1f78b4", "#b2df8a", "#33a02c", "#fb9a99",
            "#e31a1c", "#fdbf6f", "#ff7f00", "#cab2d6", "#6a3d9a"]

colors = []
for i in range(len(timeline_data)) :
    colors.append(colormap[timeline_data.loc[i , 'entity_type_id']])


source = ColumnDataSource(data=dict(
    xname=timeline_data.date.astype(str).tolist(),
    yname=timeline_data.entity_name.tolist() ,
    colors=colors,
    alphas= timeline_data.percent_right.tolist() ,
    type_fac = timeline_data.entity_type ,
    #count=counts.flatten(),
))


TOOLS = "hover,save,pan,box_zoom,wheel_zoom,reset"

p = figure(title="Monthly Reporting by facility",
           x_axis_location="above", tools=TOOLS, 
           x_range=sorted(list(set(timeline_data.date.astype(str).tolist()))), 
           y_range=result['entity_name'].astype(str).tolist())

p.plot_width = 800
p.plot_height = 1200
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = np.pi/3

p.rect('xname', 'yname', 0.9, 0.9,  source=source,
       color='colors', alpha='alphas', line_color=None,
       hover_line_color='black', hover_color='colors')
p.select_one(HoverTool).tooltips = [
    ('Fosa', '@yname') ,
    ('Period' , '@xname') ,
    ('% Right' , '@alphas')
]


show(p)

Loading BokehJS ...

We can see data has been entered since March 2012 in a group of facilities, and is entered regularly until november 2016. A second group of facilities start reporting in July 2015 and appear to stop reporting one year later. Some facilites start reporting around April 2014 and have been reporting since then.

The shading shows the percentage of indicators that had to be corrected after validation. We note a time pattern in this correction, as more indicators appear to have to be corrected at the end of quarters. This may be due to a lesser verification of previous months during supervision. This may pose a problem for modelling data error, as data error would be underestimated in two thirds of the data. Meanwhile, this could be use to define a "quality of supervision" metric.

## Amount Spent

In [18]:
order_zones = pd.DataFrame({'order':np.argsort(data.geozone_parentid.unique()) , 'zone_id':data.geozone_parentid.unique()})

In [19]:
zones = ['All'] + sorted(data.parent_geozone_name.unique().tolist())
zone_color_dico = {}
for u in range(1 , len(zones)) :
    zone_color_dico[zones[u]] = colormap[u]

In [20]:
data['parent_geozone_name_color'] = ""
for i in sorted(data.parent_geozone_name.unique().tolist()) :
    data.loc[data.parent_geozone_name == i , 'parent_geozone_name_color'] = zone_color_dico[i]

In [21]:
def received_amounts(data):
    return data.indicator_montant.sum(skipna = True)

amount_fac = data.groupby(['entity_id' , 'date' , 'entity_type' , 'entity_name' , 'parent_geozone_name' , 'parent_geozone_name_color' ,'geozone_parentid']).apply(received_amounts)

amount_fac = amount_fac[amount_fac < 50000000]

amount_fac = amount_fac.reset_index()
amount_fac.columns = ['entity_id' , 'date' , 'entity_type' , 'entity_name' ,'parent_geozone_name' , 'parent_geozone_name_color' , 'geozone_parentid' , 'value']

In [22]:
def make_multiline_data(data):
    xs = [data.date.tolist()]
    ys = [data.value.tolist()]
    color = [data.parent_geozone_name_color.tolist()]
    return pd.DataFrame([[xs[0] , ys[0]]])

amount_fac = amount_fac.groupby(['entity_id' ,'entity_type' , 'entity_name' , 'parent_geozone_name' , 'parent_geozone_name_color']).apply(make_multiline_data)
amount_fac = amount_fac.reset_index()
amount_fac.columns = ['entity_id' ,'entity_type' , 'entity_name' , 'parent_geozone_name' , 'parent_geozone_name_color', 'level' , 'xs' , 'ys']

In [25]:
output_notebook()

source = ColumnDataSource(data=dict(xs = [] , ys = [] , color = [], facility_name = [] , legend = []))

hover = HoverTool(tooltips=[
    ("Facility", "@facility_name") ,
    ("Zone" , "@legend")
])

TOOLS = "hover,save,pan,box_zoom,wheel_zoom,reset"
p = figure(x_axis_type="datetime",  plot_height=600, plot_width=700, title="Evolution of amounts paid" , tools=TOOLS)
p.grid.grid_line_alpha=0.3
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Amount Paid'
p.multi_line('xs' , 'ys',  source = source , color='color' , alpha=0.4)

def update(Zones = "All"):
    df = amount_fac
    if (Zones != "All"):
        df = df[df.parent_geozone_name.str.contains(Zones)==True]
    source.data = dict(
        color = df['parent_geozone_name_color'].tolist() ,
        xs=df['xs'].tolist(),
        ys=df['ys'].tolist(),
        facility_name = df['entity_name'].tolist(),
        legend = df['parent_geozone_name'].tolist(),
    )
    push_notebook()

show(p, notebook_handle=True)
update()

Loading BokehJS ...

NameError: name 'output_file' is not defined

In [24]:
interact(update, Zones=zones)

<function __main__.update>

## Comparing errors

In [26]:
def correct_amount(data) :
    data.claimed_montant = sum(data.indicator_claimed_value * data.indicator_tarif)
    data.difference_montant = sum(data.indicator_montant - data.claimed_montant)
    return pd.DataFrame([[data.claimed_montant.tolist() , data.difference_montant.tolist()]])

In [27]:
verif_data = data.groupby(['entity_id' , 'entity_name' , 'parent_geozone_name' , 'indicator_id' , 'date']).apply(correct_amount)

In [28]:
verif_data_report = data.groupby(['entity_id' , 'entity_name' , 'parent_geozone_name' ,'date']).apply(correct_amount)

In [30]:
verif_data_report = verif_data_report.reset_index()

verif_data_report.columns = ['entity_id' , 'entity_name' , 'parent_geozone_name' , 'date' , 'level_5', 
                      'claimed_montant' , 'difference_montant']

In [31]:
#verif_data_report = verif_data_report[(np.abs(verif_data.difference_montant) < 1000000) & (verif_data.difference_montant != 0)]

verif_data_report = verif_data_report[(np.abs(verif_data_report.difference_montant) < 500000000)]

In [33]:
np.median(verif_data_report.difference_montant)

-11822210.0